# Семинар 3. Знакомство с sklearn

In [68]:
import pandas as pd
import numpy as np

Мы будем работать с [данными клиентов банка](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data) (задача кредитного скоринга). Для целей семинара данные были преобразованы в немного другой формат.

Значение признаков:
* account: банковский счет (-1: отриц. баланс, 0: нет счета, 1: до 200 ед., 2: более 200 ед.)
* duration: на какой период запрашивают кредит
* credit_history: рейтинг по кредитной истории (от 0 - отличная кр. история до 4 - критическая ситуация с кредитами)
* amount: на какую сумму запрашивают кредит
* savings: сберегательный счет (0: нет, 1: < 100, 2: 100 <= ... < 500, 3: 500 <= ... < 1000, 4:  >= 1000)
* employment: срок работы на текущей позиции (0: не работает, 1: до 1 года, 2: от 1 до 4 лет, 3: от 4 до 7 лет, 4: более 7 лет)
* guarantors: 1 - есть поручители, 0 - нет
* residence:  сколько лет резидент
* age: возраст, деленный на 100
* credits_number: число кредитов
* job: 0: не работает, 1: неквалифицированный, 2: квалифицированный специалист, 3: высокая должность или бизнес
* maintenance_people: число людей, которых обеспечивает заемщик
* telephone: указан ли телефон (1: да, 0: нет)
* foreign: 1: иностранец, 0: нет
* real_estate: 1: есть недвижимость, 0: нет недвижимости
* life_insurance: 1: оформлено страхование жизни, 0, нет страховки
* car: 1: есть автомобиль, 0: нет автомобиля
* housing_rent: 1: платит за съем жилья, 0: не платит за съем жилья
* sex: пол - 1: муж., 0: жен.
* purpose: на какую цель запрашивают кредит (из нескольких вариантов)
* target: 1: кредит выдан, 0: в кредите отказано

Требуется решить задачу предсказания значения в последнем столбце, то есть задачу бинарной классификации.

In [42]:
tab = pd.read_excel("https://github.com/iad-24/seminars/raw/master/materials/scoring.xls")

In [43]:
tab.head()

,account,duration,credit_history,amount,savings,employment,guarantors,residence,age,credits_number,...,maintenance_people,telephone,foreign,real_estate,life_insurance,car,housing_rent,sex,purpose,target
0,1.0,0.18,2.0,0.13110,2.0,2.0,1.0,4.0,0.34,1.0,...,1,0,1,0,0,1,0,1,business,1
1,-1.0,0.18,0.0,0.15520,1.0,3.0,1.0,1.0,0.31,1.0,...,1,1,1,0,1,0,0,1,business,1
2,-1.0,0.12,2.0,0.04510,1.0,3.0,1.0,4.0,0.21,1.0,...,1,0,1,0,1,0,1,1,retraining,0
3,-1.0,0.30,0.0,0.40360,0.0,1.0,1.0,3.0,0.25,3.0,...,1,0,1,0,0,1,0,1,business,1
4,0.0,0.36,2.0,0.09095,1.0,2.0,1.0,4.0,0.37,1.0,...,1,1,1,0,0,0,0,1,education,0


In [44]:
tab.columns

Index([u'account', u'duration', u'credit_history', u'amount', u'savings',
       u'employment', u'guarantors', u'residence', u'age', u'credits_number',
       u'job', u'maintenance_people', u'telephone', u'foreign', u'real_estate',
       u'life_insurance', u'car', u'housing_rent', u'sex', u'purpose',
       u'target'],
      dtype='object')

Выделяем матрциу объекты-признаки и вектор правильных ответов:

In [45]:
X = tab[tab.columns[:-2]]
y = tab[tab.columns[-1]]

In [46]:
X.shape, y.shape

((600, 19), (600,))

Смотрим, есть ли у нас пропущенные значения:

In [47]:
X.isnull().any()

account                True
duration               True
credit_history         True
amount                 True
savings                True
employment             True
guarantors             True
residence              True
age                    True
credits_number         True
job                    True
maintenance_people    False
telephone             False
foreign               False
real_estate           False
life_insurance        False
car                   False
housing_rent          False
sex                   False
dtype: bool

Удаляем пропущенные значения:

In [48]:
from sklearn.preprocessing import Imputer

In [49]:
imputer = Imputer()
X_new = imputer.fit_transform(X)

Проверяем, что на месте пропущенных значений теперь числа:

In [74]:
X_new[X.values!=X_new]

array([ 0.35115578,  0.90284757,  0.35115578,  0.17386064,  1.2006689 ,
        0.90284757,  1.36454849,  0.08347245,  0.17386064,  2.34949833,
        0.21874791,  1.90468227,  0.17386064,  0.35115578,  0.90284757,
        2.34949833,  2.82971619,  1.36454849,  2.41068447,  1.2006689 ,
        1.90468227])

Делим выборку на обучение и контроль:

In [50]:
from sklearn.model_selection import train_test_split

In [75]:
X_train, X_test, y_train, y_test = \
                           train_test_split(X_new, y,
                                            test_size=0.3)

In [76]:
X_train.shape, X_test.shape

((420, 19), (180, 19))

Обучаем константный алгоритм:

In [77]:
from sklearn.dummy import DummyClassifier

In [78]:
clf = DummyClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

Вычисляем качество:

In [79]:
from sklearn.metrics import accuracy_score

In [80]:
accuracy_score(y_test, y_pred)

0.47222222222222221

In [81]:
(y_test==y_pred).mean()

0.47222222222222221

Этот классификатор показывает, ниже какого качества опускаться нельзя :)